### lxml Implementation

In [1]:
import json
from pathlib import Path
from lxml import etree as et

file_read = Path.cwd() / "files" / "processes" / "AccountProcess.process"

with open(file_read, "r+") as f:
    tree = et.parse(f)

root = tree.getroot()
root

<Element {http://xmlns.tibco.com/bw/process/2003}ProcessDefinition at 0x1066bc040>

### Remove Namespaces using XSLT Transformation

In [2]:
xslt_doc = Path.cwd() / "files" / "processes" / "xslt_doc.xml"
with open(xslt_doc, "r+") as f:
    xslt = et.parse(f)

transform = et.XSLT(xslt)
result = transform(root)

transformed_doc = Path.cwd() / "files" / "processes" / "transformed_doc.xml"
result.write(transformed_doc)

### Generate Tibco Mappings as Python Dictionary and dump into JSON file

In [3]:
def get_mapping(mapper, write_path):
    mappings = mapper.find("./inputBindings/root/")
    my_dict = {}

    slicer = len(mappings.xpath("ancestor::*"))

    for var in mappings.xpath(".//value-of"):
        # Build the full_path from inputBindings with slicing logic
        ancestors_list = var.xpath("ancestor::*")  # excludes self
        full_path = "/".join(
            ancestor.tag for ancestor in ancestors_list[slicer:] if ancestor.tag not in ("if"))

        my_dict[full_path] = var.attrib["select"].strip('"')

    with open(write_path, "w", encoding="utf-8") as json_file:
        json.dump(my_dict, json_file, indent=4, ensure_ascii=False)

In [4]:
activities = result.findall("./activity")

mapper_list = []
for activity in activities:
    if activity.xpath(".//resourceType='ae.activities.MapperActivity'"):
        mapper_list.append(activity)

dict_path = Path.cwd() / "files" / "processes" / "mapping_dictionary.json"
if len(mapper_list) == 1:
    get_mapping(mapper_list[0], dict_path)